In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pickle



In [2]:
user_profiles = pd.read_csv('user_profiles.csv')
workoutfeedback = pd.read_csv('workoutfeedback.csv')

#Data Preprocessing


##User Profiles

In [3]:
def preprocess_and_encode_user_data(user_profiles, given_user_df):
    scaler = MinMaxScaler()
    columns_to_scale = ['Age', 'Height (cm)', 'Weight (kg)', 'BMI']
    user_profiles[columns_to_scale] = scaler.fit_transform(user_profiles[columns_to_scale])
    given_user_df[columns_to_scale] = scaler.transform(given_user_df[columns_to_scale])
    combined_df = pd.concat([user_profiles, given_user_df], ignore_index=True)

    le = LabelEncoder()
    categorical_columns = ['Gender', 'Primary Fitness Goal', 'Secondary Fitness Goal', 'Preferred Activity Type', 'Workout Intensity']

    for col in categorical_columns:
        le.fit(combined_df[col])
        combined_df[col] = le.transform(combined_df[col])

    encoded_user_profiles = combined_df.iloc[:-1].copy()
    encoded_given_user = combined_df.iloc[-1:].copy()

    return encoded_user_profiles, encoded_given_user


In [4]:
given_user = {
    'uid': 1012,
    'Age': 29,
    'Gender': 'Female',  # This is a string, we need to encode it
    'Height (cm)': 164,
    'Weight (kg)': 91,
    'BMI': 33.83,
    'Primary Fitness Goal': 'Endurance',  # String, needs encoding
    'Secondary Fitness Goal': 'Stress reduction',  # String, needs encoding
    'Preferred Activity Type': 'HIIT',  # String, needs encoding
    'Workout Intensity': 'Beginner'  # String, needs encoding
}

given_user_df = pd.DataFrame([given_user])
encoded_user_profiles,encoded_given_user =  preprocess_and_encode_user_data(user_profiles,given_user_df)

#Collaborative Filtering to find similar Users

In [5]:
def collaborative_filtering(encoded_user_profiles, encoded_given_user, top_n=5):
    all_users_df = pd.concat([encoded_user_profiles, encoded_given_user], ignore_index=True)
    similarity_matrix = cosine_similarity(all_users_df)
    similarity_scores = similarity_matrix[-1][:-1]
    most_similar_user_indices = np.argsort(similarity_scores)[::-1][:top_n]
    top_n_similarity_scores = similarity_scores[most_similar_user_indices]
    user_ids = list(encoded_user_profiles.iloc[most_similar_user_indices]['uid'])
    return user_ids

In [6]:
user_ids = collaborative_filtering(encoded_user_profiles, encoded_given_user, top_n=5)

#Finding Exercises that the similar users liked

In [7]:
def generate_exercise_plan(workoutfeedback,similar_users_ids, top_n=5):
    similar_users_feedback = workoutfeedback[workoutfeedback['uid'].isin(similar_users_ids)]
    exercise_plan = similar_users_feedback.groupby('exid').agg({
        'frequency': 'sum',
        'rating': 'mean',
        'motivation_level': 'mean'
    }).reset_index()
    exercise_plan_sorted = exercise_plan.sort_values(by=['frequency', 'rating'], ascending=[False, False])
    recommended_exercises = exercise_plan_sorted.head(top_n)
    return recommended_exercises



In [8]:
recommended_exercises = generate_exercise_plan(workoutfeedback,user_ids, top_n=5)

print("Recommended Exercises for Given User:")
print(recommended_exercises)


Recommended Exercises for Given User:
   exid       frequency  rating  motivation_level
2  1728    Twice a week     4.0               5.0
4  2407    Twice a week     1.0               1.0
5  2563           Daily     5.0               3.0
3  2399           Daily     3.0               5.0
0    12  3 times a week     3.0               4.0


In [9]:
def create_weekly_schedule(exercises):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    schedule = {day: [] for day in days}
    for i, row in exercises.iterrows():
        exid = row['exid']
        frequency = row['frequency']
        if frequency == 'Daily':
            for day in days:
                schedule[day].append(exid)
        elif frequency == '5 times a week':
            selected_days = np.random.choice(days, 5, replace=False)
            for day in selected_days:
                schedule[day].append(exid)
        elif frequency == '3 times a week':
            selected_days = np.random.choice(days, 3, replace=False)
            for day in selected_days:
                schedule[day].append(exid)
        elif frequency == 'Twice a week':
            selected_days = np.random.choice(days, 2, replace=False)
            for day in selected_days:
                schedule[day].append(exid)
    return schedule

In [10]:
weekly_schedule = create_weekly_schedule(recommended_exercises)

In [11]:
for i in weekly_schedule:
  print(f"{i}: {weekly_schedule[i]}")

Monday: [2563, 2399, 12]
Tuesday: [1728, 2563, 2399, 12]
Wednesday: [2407, 2563, 2399, 12]
Thursday: [1728, 2563, 2399]
Friday: [2563, 2399]
Saturday: [2407, 2563, 2399]
Sunday: [2563, 2399]


#Final Model

In [12]:
def FinalRecommenderSystem_initial(given_user_df,user_profiles,workoutfeedback):
    encoded_user_profiles,encoded_given_user =  preprocess_and_encode_user_data(user_profiles,given_user_df)
    user_ids = collaborative_filtering(encoded_user_profiles, encoded_given_user, top_n=5)
    recommended_exercises = generate_exercise_plan(workoutfeedback, user_ids, top_n=5)
    weekly_schedule = create_weekly_schedule(recommended_exercises)
    return weekly_schedule

In [13]:
FinalRecommenderSystem_initial(given_user_df,user_profiles,workoutfeedback)

{'Monday': [2563, 2399],
 'Tuesday': [2563, 2399],
 'Wednesday': [2563, 2399, 12],
 'Thursday': [2407, 2563, 2399, 12],
 'Friday': [2563, 2399, 12],
 'Saturday': [1728, 2563, 2399],
 'Sunday': [1728, 2407, 2563, 2399]}

In [14]:
def FinalRecommenderSystem(given_user_df,user_profiles,workoutfeedback,n_users,n_exercises):
    scaler = MinMaxScaler()
    columns_to_scale = ['Age', 'Height (cm)', 'Weight (kg)', 'BMI']
    user_profiles[columns_to_scale] = scaler.fit_transform(user_profiles[columns_to_scale])
    given_user_df[columns_to_scale] = scaler.transform(given_user_df[columns_to_scale])
    combined_df = pd.concat([user_profiles, given_user_df], ignore_index=True)

    le = LabelEncoder()
    categorical_columns = ['Gender', 'Primary Fitness Goal', 'Secondary Fitness Goal', 'Preferred Activity Type', 'Workout Intensity']

    for col in categorical_columns:
        le.fit(combined_df[col])
        combined_df[col] = le.transform(combined_df[col])

    encoded_user_profiles = combined_df.iloc[:-1].copy()
    encoded_given_user = combined_df.iloc[-1:].copy()

    all_users_df = pd.concat([encoded_user_profiles, encoded_given_user], ignore_index=True)
    similarity_matrix = cosine_similarity(all_users_df)
    similarity_scores = similarity_matrix[-1][:-1]
    most_similar_user_indices = np.argsort(similarity_scores)[::-1][:n_users]
    top_n_similarity_scores = similarity_scores[most_similar_user_indices]
    user_ids = list(encoded_user_profiles.iloc[most_similar_user_indices]['uid'])

    similar_users_feedback = workoutfeedback[workoutfeedback['uid'].isin(user_ids)]
    exercise_plan = similar_users_feedback.groupby('exid').agg({
        'frequency': 'sum',
        'rating': 'mean',
        'motivation_level': 'mean'
    }).reset_index()
    exercise_plan_sorted = exercise_plan.sort_values(by=['frequency', 'rating'], ascending=[False, False])
    recommended_exercises = exercise_plan_sorted.head(n_exercises)

    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    schedule = {day: [] for day in days}
    for i, row in recommended_exercises.iterrows():
        exid = row['exid']
        frequency = row['frequency']
        if frequency == 'Daily':
            for day in days:
                schedule[day].append(exid)
        elif frequency == '5 times a week':
            selected_days = np.random.choice(days, 5, replace=False)
            for day in selected_days:
                schedule[day].append(exid)
        elif frequency == '3 times a week':
            selected_days = np.random.choice(days, 3, replace=False)
            for day in selected_days:
                schedule[day].append(exid)
        elif frequency == 'Twice a week':
            selected_days = np.random.choice(days, 2, replace=False)
            for day in selected_days:
                schedule[day].append(exid)
    return schedule

In [ ]:
with open('recommender_model.pkl', 'wb') as file:
    pickle.dump(FinalRecommenderSystem, file)